# Einführung in das Clustering mit k-Means

## Laden der Bibliotheken

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score

from ipywidgets import interact
import ipywidgets as widgets

import seaborn as snss

style = {'description_width': '150px'}
layout = widgets.Layout(width='400px')

## Vier synthetische Cluster

### Generierung und Visualisierung der Daten

Um die Funktionsweise des Clusterings zu prüfen, beginnen wir zuerst mit künstlich generierten Daten. Diese werden in vier Clustern verteilt, die um die Punkte (-1, -1), (-1, 1), (1, -1), (1, 1) zentriert sind

In [ ]:
n = 100

data = np.concatenate(
    [np.random.multivariate_normal([x, y], np.diag([.1, .1]), size=n)
     for x in [-1, 1]
     for y in [-1, 1]]
)

In [ ]:
df = pd.DataFrame(data, columns=['x', 'y'])
_ = df.plot(kind='scatter', x='x', y='y', s=40, figsize=(6, 6))

### Clustering mit k-Means

Wir betrachten nun, welche Cluster das populäre k-Means-Verfahren identifiziert. Den Parameter k stellen wir über einen Schieberegler ein.

In [ ]:
def fit_and_plot_clustering(df, clusterer, print_number=False):
    df = df.copy()
    df['clusterid'] = clusterer.fit_predict(df[['x', 'y']])
    n_clusters = df['clusterid'].max() + 1
    n_outliers = np.sum(df['clusterid'] == -1)
    if print_number:
        print(f'number of clusters: {n_clusters}\nnumber of outliers: {n_outliers}')
    cmap = plt.get_cmap('Set1', n_clusters+1)
    ax = df.plot(kind='scatter', x='x', y='y', c='clusterid', cmap=cmap, s=40, colorbar=False, figsize=(6, 6))
    ax.grid()

def plot_kmeans(k = 3):
    kmeans = KMeans(n_clusters=k)
    fit_and_plot_clustering(df, clusterer=kmeans)


_ = interact(
    plot_kmeans,
    k=widgets.SelectionSlider(
        options=range(1,10),
        layout=layout,
        style=style,
        description='Number of clusters (k)',
        orientation='horizontal'
    )
)

Wir berechnen nun die zugehörige Silhouetten-Kurve. Wie man sehen kann, wird der höchste Wert bei 4, also der korrekten Anzahl der Cluster, erreicht.

In [ ]:
sil_values = []
for k in range(2,20):
    kmeans = KMeans(n_clusters=k)
    clust = kmeans.fit_predict(df[['x', 'y']])
    sil_values.append(silhouette_score(df[['x', 'y']], clust))

fig, ax = plt.subplots()
ax.plot(range(2,20),sil_values)
ax.set_xlabel('k')
ax.set_xticks(range(2, 20, 2))
ax.set_ylabel('Silhouette value');

## Iris-Datenset

Als nächstes überprüfen wir die Funktionsweise von k-Means an einem echten Datensatz, dem berühmten Iris-Datensatz von Ronald Fisher (https://en.wikipedia.org/wiki/Iris_flower_data_set). Hierbei handelt es sich um Messungen von Blättern für unterschiedliche Arten aus der Gattung Iris (Schwertlilien).

Dieses wird zuerst aus der Datei "iris.csv" eingelesen und dann visualisiert. Dabei sind auf der X-Achse die Breite der Kelchblätter und auf der Y-Achse die Länge der Blütenblätter aufgetragen.

In [ ]:
dataset_path = os.environ['DATASET_PATH']
df = pd.read_csv(dataset_path + '/Iris/iris.csv'). \
        rename({'sepal width (cm)': 'x', 'petal length (cm)': 'y'}, axis=1)
_ = df.plot(kind='scatter', x='x', y='y', s=40, figsize=(6, 6))

### Clustering mit k-Means

In [ ]:
_ = interact(
    plot_kmeans,
    k=widgets.SelectionSlider(
        options=range(1, 10),
        description='Number of clusters (k)',
        layout=layout,
        style=style,
        orientation='horizontal'
    )
)

Auch hier erzeugen wir ein Silhouetten-Bild. Das Maximum liegt hier bei zwei Clustern. Dieser Wert scheint auf Basis der Daten auch visuell plausibel.

In [ ]:
sil_values = []
for k in range(2,20):
    kmeans = KMeans(n_clusters=k)
    clust = kmeans.fit_predict(df[['x', 'y']])
    sil_values.append(silhouette_score(df[['x', 'y']], clust))

fig, ax = plt.subplots()
ax.plot(range(2,20),sil_values)
ax.set_xlabel('k')
ax.set_xticks(range(2,20, 2))
ax.set_ylabel('Silhouette value');

## Die Auflösung

Tatsächlich entstammen die Daten jedoch *drei* verschiedenen Arten. Dies sehen wir, wenn wir die Spalte mit der Art hinzunehmen und die Punkte farbig kennzeichnen. Man sieht, dass die rote Art gut getrennt ist, jedoch die organene und graue Art ineinander übergehen. Diese Schwierigkeit bei der Separierung ist in der Praxis häufig anzutreffen.

In [ ]:
_ = df.plot(kind='scatter',
            x='x',
            y='y',
            s=40,
            c='class',
            cmap=plt.get_cmap('Set1'),
            colorbar=False,
            figsize=(6, 6)
           )